In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pandas_datareader as web
import seaborn as sns
from datetime import datetime
import yfinance as yf
yf.pdr_override()

In [12]:
# Lista de tickers
tickers = ["PETR4.SA", 'VALE3.SA', 'ITUB4.SA', 'BBAS3.SA', 'BBDC3.SA']

# Baixando os dados dos tickers
carteira_compra = yf.download(tickers, start="2002-01-01", end="2023-12-31")['Close']
carteira = yf.download(tickers, start="2002-01-01", end="2023-12-31")['Adj Close']

# Baixando os dados do índice Ibovespa
ibov = yf.download('^BVSP', start="2002-01-01", end="2023-12-31")['Adj Close']

[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  5 of 5 completed
[*********************100%***********************]  1 of 1 completed


In [13]:
resultados_vendacoberta = pd.read_json(r'C:\Users\cptfr\Desktop\TCC_B3_OPCOES\estrategias\resultados_vendacoberta.json')
resultados_vendacoberta['Data_Vencimento'] = pd.to_datetime(resultados_vendacoberta['Data_Vencimento'], format='%Y%m%d')
print(resultados_vendacoberta)

    Data_Entrada Data_Vencimento  Tipo     Ativo  ResultadosTotais
0       20041001      2004-11-16  CALL  PETR4.SA              0.56
1       20050919      2005-10-17  CALL  PETR4.SA              1.55
2       20070102      2007-02-12  CALL  PETR4.SA              0.03
3       20080519      2008-06-16  CALL  PETR4.SA              0.15
4       20090918      2009-10-19  CALL  BBDC3.SA              1.00
5       20130102      2013-02-18  CALL  BBAS3.SA              0.38
6       20131031      2013-12-16  CALL  BBAS3.SA              0.10
7       20140722      2014-09-15  CALL  BBAS3.SA              0.35
8       20140828      2014-10-20  CALL  BBAS3.SA              0.50
9       20161021      2016-12-19  CALL  PETR4.SA              1.00
10      20170921      2017-11-21  CALL  BBAS3.SA              0.55
11      20180122      2018-03-19  CALL  PETR4.SA              0.05
12      20180511      2018-06-18  CALL  VALE3.SA              0.23
13      20180201      2018-03-19  CALL  BBAS3.SA              

In [14]:
acoes_num = (10000/ carteira_compra.iloc[0]).astype(int)
acoes_num
#acoes_num['PETR4.SA']

BBAS3.SA    5665
BBDC3.SA    5200
ITUB4.SA    3431
PETR4.SA    1564
VALE3.SA    2307
Name: 2002-01-01 00:00:00, dtype: int32

In [15]:
carteira

,BBAS3.SA,BBDC3.SA,ITUB4.SA,PETR4.SA,VALE3.SA
Date,,,,,
2002-01-01,0.263994,1.026873,1.617726,1.571675,1.896705
2002-01-02,0.268730,1.026873,1.691356,1.574747,1.860231
2002-01-03,0.272968,1.034691,1.672880,1.585502,1.889410
2002-01-04,0.266736,1.030782,1.645395,1.575669,1.929533
2002-01-07,0.260753,1.034691,1.627103,1.610083,1.929533
...,...,...,...,...,...
2023-12-21,26.554211,14.914568,31.443796,33.934216,73.674408
2023-12-22,26.627575,14.951639,31.942444,34.260597,73.119240
2023-12-26,26.872135,15.011247,32.268486,34.810783,73.377686


In [16]:
# Caminho do arquivo JSON com os dados da taxa Selic
caminho_do_arquivo = r'C:\Users\cptfr\Desktop\TCC_B3_OPCOES\dadosselicdiario.json'

# Lendo o arquivo JSON e transformando em DataFrame para manipulação de dados
selic = pd.read_json(caminho_do_arquivo)

# Converte a coluna 'data' do formato string para datetime, facilitando manipulações de data
selic['data'] = pd.to_datetime(selic['data'], format='%d/%m/%Y')

# Renomeia a coluna 'data' para 'Date' para padronização em inglês
selic.rename(columns={'data':'Date'}, inplace=True)

# Define a coluna 'Date' como índice do DataFrame, otimizando buscas e filtragens por data
selic.set_index('Date', inplace=True)

# Cria um novo DataFrame chamado 'selic_valores' contendo apenas a coluna 'valor', agora renomeada para 'SELIC'
selic_valores = pd.DataFrame(selic['valor']).rename(columns={'valor':'SELIC'})
selic_valores.index = pd.to_datetime(selic_valores.index)
selic_valores

,SELIC
Date,
2002-01-02,0.069220
2002-01-03,0.069220
2002-01-04,0.069220
2002-01-07,0.069220
2002-01-08,0.069220
...,...
2023-12-22,0.043739
2023-12-26,0.043739
2023-12-27,0.043739


In [17]:
carteira = carteira.join(selic_valores, how='left')
carteira

,BBAS3.SA,BBDC3.SA,ITUB4.SA,PETR4.SA,VALE3.SA,SELIC
Date,,,,,,
2002-01-01,0.263994,1.026873,1.617726,1.571675,1.896705,NaN
2002-01-02,0.268730,1.026873,1.691356,1.574747,1.860231,0.069220
2002-01-03,0.272968,1.034691,1.672880,1.585502,1.889410,0.069220
2002-01-04,0.266736,1.030782,1.645395,1.575669,1.929533,0.069220
2002-01-07,0.260753,1.034691,1.627103,1.610083,1.929533,0.069220
...,...,...,...,...,...,...
2023-12-21,26.554211,14.914568,31.443796,33.934216,73.674408,0.043739
2023-12-22,26.627575,14.951639,31.942444,34.260597,73.119240,0.043739
2023-12-26,26.872135,15.011247,32.268486,34.810783,73.377686,0.043739


In [18]:
def preencher_nans(df):
    # Preenche NaNs com o valor anterior (forward fill)
    df.ffill(inplace=True)
    # Para os casos onde o primeiro valor é NaN, preenche com o valor seguinte (backward fill)
    df.bfill(inplace=True)
    return df

In [19]:
carteira = preencher_nans(carteira)

In [20]:
inicio_2002 = '2002-01-01'
fim_2002 = '2002-12-31'

valores_iniciais = {
    'BBAS3.SA': 6000,
    'BBDC3.SA': 6000,
    'ITUB4.SA': 6000,
    'PETR4.SA': 6000,
    'VALE3.SA': 6000,
    'SELIC': 20000
}
carteira_geral1 = carteira.loc[inicio_2002:fim_2002]
carteira_normalizada_2002 = pd.DataFrame()

for coluna in carteira_geral1.columns:
    carteira_normalizada_2002[coluna] = (carteira_geral1[coluna] / carteira_geral1[coluna].iloc[0])* valores_iniciais[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2002['saldo'] = carteira_normalizada_2002.sum(axis=1)
valores_ult_2002_selic = carteira_normalizada_2002['saldo'][-1] * 0.4
valores_ult_2002 = (carteira_normalizada_2002['saldo'][-1] * 0.6) / 5
valores_2002 = {
    'BBAS3.SA': valores_ult_2002,
    'BBDC3.SA': valores_ult_2002,
    'ITUB4.SA': valores_ult_2002,
    'PETR4.SA': valores_ult_2002,
    'VALE3.SA': valores_ult_2002,
    'SELIC': valores_ult_2002_selic
}
print(carteira_normalizada_2002)
print(valores_ult_2002)

               BBAS3.SA     BBDC3.SA     ITUB4.SA     PETR4.SA      VALE3.SA  \
Date                                                                           
2002-01-01  6000.000000  6000.000000  6000.000000  6000.000000   6000.000000   
2002-01-02  6107.649835  6000.000000  6273.086067  6011.730438   5884.617016   
2002-01-03  6203.969106  6045.680334  6204.560211  6052.785607   5976.923102   
2002-01-04  6062.323517  6022.840167  6102.622982  6015.248296   6103.846231   
2002-01-07  5926.347274  6045.680334  6034.779785  6146.625474   6103.846231   
...                 ...          ...          ...          ...           ...   
2002-12-25  5575.074070  5131.974040  5795.787308  5719.648166  11551.154609   
2002-12-26  5552.408541  5071.059382  5729.307218  5618.768489  11769.234033   
2002-12-27  5495.752337  5482.225925  5753.170653  5472.139375  11769.234033   
2002-12-30  5552.408541  5824.864306  5846.932777  5442.813279  11873.079510   
2002-12-31  5552.408541  5824.864306  58

In [22]:
inicio_2003 = '2003-01-01'
fim_2003 = '2003-12-31'

carteira_geral2 = carteira.loc[inicio_2003:fim_2003]

carteira_normalizada_2003 = pd.DataFrame()

for coluna in carteira_geral1.columns:
    carteira_normalizada_2003[coluna] = (carteira_geral2[coluna] / carteira_geral2[coluna].iloc[0])* valores_2002[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2003['saldo'] = carteira_normalizada_2003.sum(axis=1)
print(len(carteira_normalizada_2003.columns))
valores_ult_2003 = (carteira_normalizada_2003['saldo'][-1] * 0.6) / 5
valores_ult_2003_selic = carteira_normalizada_2003['saldo'][-1] * 0.4
valores_2003 = {
    'BBAS3.SA': valores_ult_2003,
    'BBDC3.SA': valores_ult_2003,
    'ITUB4.SA': valores_ult_2003,
    'PETR4.SA': valores_ult_2003,
    'VALE3.SA': valores_ult_2003,
    'SELIC': valores_ult_2003_selic
}
print(carteira_normalizada_2003)
print(valores_ult_2003)

7
                BBAS3.SA     BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                
2003-01-01   7205.314554  7205.314554   7205.314554   7205.314554   
2003-01-02   7234.727424  6913.340088   7058.262993   7314.016415   
2003-01-03   7227.374756  6856.825232   7152.789952   7282.960806   
2003-01-06   7278.844312  6913.340088   7406.974790   7624.591783   
2003-01-07   7315.603697  7148.807022   7226.311101   7450.668806   
...                  ...          ...           ...           ...   
2003-12-25  17535.386473  9249.181054  12057.437985  11786.285565   
2003-12-26  17278.061548  9381.038712  11889.384974  11634.100551   
2003-12-29  17233.940266  9428.136148  12099.870232  11870.141975   
2003-12-30  17645.677373  9503.488428  12057.867331  11863.930612   
2003-12-31  17645.677373  9503.488428  12057.867331  11863.930612   

                VALE3.SA         SELIC         saldo  
Date                                         

In [23]:
#PRIMEIRA ENTRADA
inicio_2004 = '2004-01-01'
fim_pre_2004 = '2004-11-16'

carteira_entrada_1 = carteira.loc[inicio_2004:fim_pre_2004]
carteira_normalizada_20041 = pd.DataFrame()

for coluna in carteira_entrada_1.columns:
    carteira_normalizada_20041[coluna] = (carteira_entrada_1[coluna] / carteira_entrada_1[coluna].iloc[0])*valores_2003[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20041['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[0]
carteira_normalizada_20041['saldo'] = carteira_normalizada_20041.sum(axis=1)
print(carteira_normalizada_20041)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2004-01-01   9513.219360   9513.219360   9513.219360   9513.219360   
2004-01-02   9394.303017   9475.509387   9272.896484   9861.867585   
2004-01-05  10167.254039   9833.778288   9944.124729  10434.655576   
2004-01-06  10543.825462   9579.222596   9944.124729  10982.537430   
2004-01-07  10286.165643   9409.505289  10060.145219  10866.731337   
...                  ...           ...           ...           ...   
2004-11-10  12287.911618   9970.495703  11535.186852  11854.167170   
2004-11-11  12212.593163   9829.064757  11455.633187  11897.745662   
2004-11-12  12327.544081  10039.081246  11502.042071  11928.876883   
2004-11-15  12327.544081  10039.081246  11502.042071  11928.876883   
2004-11-16  12684.295956   9899.785836  11303.157908  12244.383893   

                VALE3.SA         SELIC         saldo  
Date                              

In [24]:
inicio_2004_2 = '2004-11-16'
fim_2004 = '2004-12-31'

carteira_geral3 = carteira.loc[inicio_2004_2:fim_2004]
carteira_normalizada_2004 = carteira_geral3 / carteira_geral3.iloc[0]
for coluna in carteira_normalizada_2004.columns:
    if coluna != 'saldo':
        carteira_normalizada_2004[coluna] *= carteira_normalizada_20041[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação


carteira_normalizada_2004['saldo'] = carteira_normalizada_2004.sum(axis=1)


valores_ult_2004 = (carteira_normalizada_2004['saldo'][-1] * 0.6) / 5
valores_ult_2004_selic = carteira_normalizada_2004['saldo'][-1] * 0.4
valores_2004 = {
    'BBAS3.SA': valores_ult_2004,
    'BBDC3.SA': valores_ult_2004,
    'ITUB4.SA': valores_ult_2004,
    'PETR4.SA': valores_ult_2004,
    'VALE3.SA': valores_ult_2004,
    'SELIC': valores_ult_2004_selic
}
print(carteira_normalizada_2004)
print(valores_ult_2004)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2004-11-16  12684.295956   9899.785836  11303.157908  12244.383893   
2004-11-17  12565.380560   9892.711658  11369.462946  12343.627151   
2004-11-18  12490.062105   9793.710255  11200.404960  12190.738213   
2004-11-19  12248.274417   9687.645024  11236.867486  12137.093574   
2004-11-22  12704.106027   9708.854620  11203.714108  12296.688222   
2004-11-23  12335.473038   9969.787595  10980.966081  12083.447895   
2004-11-24  12430.601564   9970.495703  11154.332235  12016.391836   
2004-11-25  12688.256643  10110.503533  11140.737992  12660.125422   
2004-11-26  12688.256643  10568.728619  11369.462946  12582.342725   
2004-11-29  12644.651172  10535.484640  11531.216563  12371.788219   
2004-11-30  12624.835414  10536.195336  11733.737435  12458.960367   
2004-12-01  12525.732933  10747.635802  11966.096519  12834.470759   
2004-12-02  12723.92

In [26]:
#ENTRADA 2
inicio_2005_1 = '2005-01-01'
fim_2005_1 = '2005-10-17'

carteira_entradaw = carteira.loc[inicio_2005_1:fim_2005_1]
carteira_normalizada_20051 = pd.DataFrame()


for coluna in carteira_entradaw.columns:
    carteira_normalizada_20051[coluna] = (carteira_entradaw[coluna] / carteira_entradaw[coluna].iloc[0])*valores_2004[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20051['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[1]
carteira_normalizada_20051['saldo'] = carteira_normalizada_20051.sum(axis=1)
print(carteira_normalizada_20051)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2005-01-03  11622.373582  11622.373582  11622.373582  11622.373582   
2005-01-04  11340.619224  11209.294060  11479.660409  11484.191789   
2005-01-05  11164.527171  10794.135232  11365.718909  11439.773991   
2005-01-06  11023.643255  10482.771808  11223.292403  11474.316806   
2005-01-07  10917.991897  10711.105365  11209.043251  11524.904674   
...                  ...           ...           ...           ...   
2005-10-10  14728.718199  21494.870647  15208.417379  16007.392622   
2005-10-11  15144.307266  22414.444698  15453.377787  16312.294573   
2005-10-13  14598.405145  21552.992503  14724.157508  15956.579779   
2005-10-14  14471.617116  21131.605248  14613.067907  15844.780418   
2005-10-17  14369.482361  21920.407021  15410.659884  18756.820859   

                VALE3.SA         SELIC          saldo  
Date                             

In [27]:
inicio_2005_2 = '2005-10-17'
fim_2005 = '2005-12-31'

carteira_geral4 = carteira.loc[inicio_2005_2:fim_2005]
carteira_normalizada_2005 = carteira_geral4 / carteira_geral4.iloc[0]
for coluna in carteira_normalizada_2005.columns:
    if coluna != 'saldo':
        carteira_normalizada_2005[coluna] *= carteira_normalizada_20051[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação


carteira_normalizada_2005['saldo'] = carteira_normalizada_2005.sum(axis=1)


valores_ult_2005 = (carteira_normalizada_2005['saldo'][-1] * 0.6) / 5
valores_ult_2005_selic = carteira_normalizada_2005['saldo'][-1] * 0.4
valores_2005 = {
    'BBAS3.SA': valores_ult_2005,
    'BBDC3.SA': valores_ult_2005,
    'ITUB4.SA': valores_ult_2005,
    'PETR4.SA': valores_ult_2005,
    'VALE3.SA': valores_ult_2005,
    'SELIC': valores_ult_2005_selic
}
print(carteira_normalizada_2005)
print(valores_ult_2005)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2005-10-17  14369.482361  21920.407021  15410.659884  18756.820859   
2005-10-18  14510.351961  21920.407021  14840.947952  17741.360750   
2005-10-19  14228.607708  21173.124928  14405.121604  18091.524943   
2005-10-20  13710.880518  20498.478065  13986.388920  17192.780002   
2005-10-21  13911.626687  21235.395904  14242.753969  17846.411987   
2005-10-24  14158.164487  21378.611743  14840.947952  18832.686433   
2005-10-25  14299.042508  21380.700545  14328.210464  18745.149232   
2005-10-26  14115.902092  21341.256373  14812.467379  18675.119472   
2005-10-27  14242.692647  20716.438824  14388.024986  18126.536525   
2005-10-28  14087.729687  21110.838761  14584.584379  18179.065442   
2005-10-31  14580.798551  21999.287768  15356.532906  18797.671552   
2005-11-01  14915.377745  23041.339656  15282.464798  19095.306829   
2005-11-03  14792.11

In [28]:
inicio_2006 = '2006-01-01'
fim_2006 = '2006-12-31'

carteira_geral4 = carteira.loc[inicio_2006:fim_2006]
carteira_normalizada_20061 = pd.DataFrame()

for coluna in carteira_geral4.columns:
    carteira_normalizada_20061[coluna] = (carteira_geral4[coluna] / carteira_geral4[coluna].iloc[0])*valores_2005[coluna]
#                                                      acessando primeira linha/primeira operação

carteira_normalizada_20061['saldo'] = carteira_normalizada_20061.sum(axis=1)
valores_ult_2006 = (carteira_normalizada_20061['saldo'][-1] * 0.6) / 5
valores_ult_2006_selic = carteira_normalizada_20061['saldo'][-1] * 0.4
valores_2006 = {
    'BBAS3.SA': valores_ult_2006,
    'BBDC3.SA': valores_ult_2006,
    'ITUB4.SA': valores_ult_2006,
    'PETR4.SA': valores_ult_2006,
    'VALE3.SA': valores_ult_2006,
    'SELIC': valores_ult_2006_selic
}
print(carteira_normalizada_20061)
print(valores_ult_2006)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2006-01-02  16029.976908  16029.976908  16029.976908  16029.976908   
2006-01-03  16381.004426  16610.589952  16470.506981  16534.998497   
2006-01-04  18604.132081  16736.817106  16939.479133  16614.953864   
2006-01-05  19157.971023  16736.817106  16825.783061  16581.289928   
2006-01-06  19910.713995  16938.766006  17422.636203  17191.514055   
...                  ...           ...           ...           ...   
2006-12-21  24598.804303  19586.871332  21486.991735  20957.500028   
2006-12-22  24181.485388  19309.171823  21259.615810  20879.474122   
2006-12-26  24337.488293  19690.378500  21486.991735  21057.198754   
2006-12-27  24805.519389  20182.620643  21884.902183  21542.656796   
2006-12-28  24961.533485  20939.955860  21998.577614  21586.008691   

                VALE3.SA         SELIC          saldo  
Date                             

In [29]:
#ENTRADA 3 etapa 1
inicio_2007_1 = '2007-01-01'
fim_2007_1 = '2007-02-12'

carteira_entrada_2007 = carteira.loc[inicio_2007_1:fim_2007_1]
carteira_normalizada_20071 = pd.DataFrame()

for coluna in carteira_entrada_2007.columns:
    carteira_normalizada_20071[coluna] = (carteira_entrada_2007[coluna] / carteira_entrada_2007[coluna].iloc[0])*valores_2006[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20071['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[2]
carteira_normalizada_20071['saldo'] = carteira_normalizada_20071.sum(axis=1)
print(carteira_normalizada_20071)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2007-01-02  18100.711080  18100.711080  18100.711080  18100.711080   
2007-01-03  17775.769975  17906.566488  18209.388720  17550.353916   
2007-01-04  18422.906505  17973.433207  18035.963663  17158.264458   
2007-01-05  17624.311116  17248.532917  17342.271830  16636.683356   
2007-01-08  18037.376062  17798.694785  17716.862613  16830.924036   
2007-01-09  17844.609523  17343.467831  17337.648985  16316.539844   
2007-01-10  17789.537884  17259.322456  17344.584452  16276.967774   
2007-01-11  18056.656403  17475.071782  17619.737288  16251.785664   
2007-01-12  18119.987471  17820.267941  17689.112348  16262.581671   
2007-01-15  18395.373320  17906.566488  17774.682960  16147.469701   
2007-01-16  18287.963871  18057.568715  17874.101715  15931.642152   
2007-01-17  18511.032980  18070.509451  18001.282729  15848.913058   
2007-01-18  18783.64

In [30]:
#ENTRADA 3 etapa 2
inicio_2007_2 = '2007-02-12'
fim_2007 = '2007-12-31'

carteira_geral_5 = carteira.loc[inicio_2007_2:fim_2007]
carteira_normalizada_2007 = carteira_geral_5 / carteira_geral_5.iloc[0]
for coluna in carteira_normalizada_2007.columns:
    if coluna != 'saldo':
        carteira_normalizada_2007[coluna] *= carteira_normalizada_20071[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2007['saldo'] = carteira_normalizada_2007.sum(axis=1)
valores_ult_2007 = (carteira_normalizada_2007['saldo'][-1] * 0.6) / 5
valores_ult_2007_selic = carteira_normalizada_2007['saldo'][-1] * 0.4
valores_2007 = {
    'BBAS3.SA': valores_ult_2007,
    'BBDC3.SA': valores_ult_2007,
    'ITUB4.SA': valores_ult_2007,
    'PETR4.SA': valores_ult_2007,
    'VALE3.SA': valores_ult_2007,
    'SELIC': valores_ult_2007_selic
}
print(carteira_normalizada_2007)
print(valores_ult_2007)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2007-02-12  18175.063060  17507.436451  17689.112348  15964.182478   
2007-02-13  18450.439691  17529.017501  17665.993326  16256.404934   
2007-02-14  19111.352033  17910.864936  18070.648194  16292.480077   
2007-02-15  18813.948063  17800.844009  17943.471978  15982.214246   
2007-02-16  18453.202491  17690.809267  17758.482611  15993.039497   
...                  ...           ...           ...           ...   
2007-12-20  25692.934363  22834.095664  21527.557443  31153.878466   
2007-12-21  25692.934363  22842.720190  21509.049270  31520.864022   
2007-12-26  25858.167716  23062.783753  21966.883648  32362.362866   
2007-12-27  25841.640957  22437.128666  21620.033534  32006.491441   
2007-12-28  25114.626846  22221.391181  21041.964403  32770.134625   

                VALE3.SA         SELIC          saldo  
Date                             

In [32]:
#Entrada 4 etapa 1
inicio_2008_1 = '2008-01-01'
fim_2008_1 = '2008-06-16'

carteira_entrada_2008 = carteira.loc[inicio_2008_1:fim_2008_1]
carteira_normalizada_20081 = pd.DataFrame()

for coluna in carteira_entrada_2008.columns:
    carteira_normalizada_20081[coluna] = (carteira_entrada_2008[coluna] / carteira_entrada_2008[coluna].iloc[0])*valores_2007[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20081['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[3]
carteira_normalizada_20081['saldo'] = carteira_normalizada_20081.sum(axis=1)
print(carteira_normalizada_20081)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2008-01-02  22270.484763  22270.484763  22270.484763  22270.484763   
2008-01-03  21934.186923  21996.799678  21850.879807  21912.101342   
2008-01-04  21680.083151  22459.883465  22004.382873  20836.972452   
2008-01-07  22195.747939  22682.168681  22004.382873  20225.177077   
2008-01-08  22651.626126  23196.211846  22209.093889  21336.144843   
...                  ...           ...           ...           ...   
2008-06-10  22644.156970  21517.894193  22708.010709  23701.984454   
2008-06-11  22195.747939  21309.315044  22388.190975  23392.831844   
2008-06-12  22128.485035  21830.755504  22899.922723  23279.475056   
2008-06-13  21523.140347  21962.835230  23162.196725  23650.457831   
2008-06-16  21971.537466  21997.602304  23219.750420  23833.529428   

                VALE3.SA         SELIC          saldo  
Date                             

In [33]:
#ENTRADA 4 etapa 2
inicio_2008_2 = '2008-06-16'
fim_2008 = '2008-12-31'

carteira_geral6 = carteira.loc[inicio_2008_2:fim_2008]
carteira_normalizada_2008 = carteira_geral6 / carteira_geral6.iloc[0]
for coluna in carteira_normalizada_2008.columns:
    if coluna != 'saldo':
        carteira_normalizada_2008[coluna] *= carteira_normalizada_20081[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2008['saldo'] = carteira_normalizada_2008.sum(axis=1)
valores_ult_2008 = (carteira_normalizada_2008['saldo'][-1] * 0.6) / 5
valores_ult_2008_selic = carteira_normalizada_2008['saldo'][-1] * 0.4
valores_2008 = {
    'BBAS3.SA': valores_ult_2008,
    'BBDC3.SA': valores_ult_2008,
    'ITUB4.SA': valores_ult_2008,
    'PETR4.SA': valores_ult_2008,
    'VALE3.SA': valores_ult_2008,
    'SELIC': valores_ult_2008_selic
}
print(carteira_normalizada_2008)
print(valores_ult_2008)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2008-06-16  21971.537466  21997.602304  23219.750420  23833.529428   
2008-06-17  22233.110394  22352.193846  23827.439503  24067.698880   
2008-06-18  21298.956083  21761.219238  22995.874749  23625.383001   
2008-06-19  21298.956083  21754.283612  22867.943139  22844.807370   
2008-06-20  20484.348764  21128.533459  21940.435650  22444.109984   
...                  ...           ...           ...           ...   
2008-12-22  11247.339279  14063.912520  17565.139027  11521.273013   
2008-12-23  10462.643129  13560.380799  16637.624902  11448.423112   
2008-12-26  10462.643129  13448.474743  16567.261587  11864.729456   
2008-12-29  10462.643129  13623.314984  16567.261587  11740.959545   
2008-12-30  10970.831613  13979.988272  16695.201161  11907.793273   

                VALE3.SA         SELIC          saldo  
Date                             

In [34]:
#ENTRADA 5 etapa 1 
inicio_2009_1 = '2009-01-01'
fim_2009_1 = '2009-10-19'

carteira_entrada_2009 = carteira.loc[inicio_2009_1:fim_2009_1]
carteira_normalizada_20091 = pd.DataFrame()

for coluna in carteira_entrada_2009.columns:
    carteira_normalizada_20091[coluna] = (carteira_entrada_2009[coluna] / carteira_entrada_2009[coluna].iloc[0])*valores_2008[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20091['BBDC3.SA'].iloc[-1] += acoes_num['BBDC3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[4]
carteira_normalizada_20091['saldo'] = carteira_normalizada_20091.sum(axis=1)
print(carteira_normalizada_20091)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2009-01-02  18483.777367  18483.777367  18483.777367  18483.777367   
2009-01-05  18828.802426  18343.616427  18528.582298  18905.572469   
2009-01-06  19863.901171  18658.985838  19290.210332  19169.201391   
2009-01-07  19420.287984  18387.417882  18182.970257  18415.985737   
2009-01-08  18976.678726  18527.577495  18291.786024  19206.858211   
...                  ...           ...           ...           ...   
2009-10-13  40497.273728  26092.116781  25415.194876  27726.380701   
2009-10-14  41787.011986  27001.044708  26119.226438  27981.822341   
2009-10-15  41335.587096  27090.167616  26013.621836  28330.134604   
2009-10-16  40290.920635  26475.307884  25767.217297  28175.323379   
2009-10-19  40484.384517  31889.155645  26062.902213  28918.407905   

                VALE3.SA         SELIC          saldo  
Date                             

In [36]:
#ENTRADA 5 etapa 2
inicio_2009_2 = '2009-10-19'
fim_2009 = '2009-12-31'

carteira_geral7 = carteira.loc[inicio_2009_2:fim_2009]
carteira_normalizada_2009 = carteira_geral7 / carteira_geral7.iloc[0]
for coluna in carteira_normalizada_2009.columns:
    if coluna != 'saldo':
        carteira_normalizada_2009[coluna] *= carteira_normalizada_20091[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2009['saldo'] = carteira_normalizada_2009.sum(axis=1)


valores_ult_2009 = (carteira_normalizada_2009['saldo'][-1] * 0.6) / 5
valores_ult_2009_selic = carteira_normalizada_2009['saldo'][-1] * 0.4
valores_2009 = {
    'BBAS3.SA': valores_ult_2009,
    'BBDC3.SA': valores_ult_2009,
    'ITUB4.SA': valores_ult_2009,
    'PETR4.SA': valores_ult_2009,
    'VALE3.SA': valores_ult_2009,
    'SELIC': valores_ult_2009_selic
}
print(carteira_normalizada_2009)
print(valores_ult_2009)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2009-10-19  40484.384517  31889.155645  26062.902213  28918.407905   
2009-10-20  39246.234533  30984.126697  25556.010750  28252.726393   
2009-10-21  39272.044383  30877.661652  25344.785611  28407.540217   
2009-10-22  40045.868481  31622.970847  25837.621250  28639.759654   
2009-10-23  39207.531542  31250.324174  25591.208743  28252.726393   
2009-10-26  38781.944002  30941.553357  25556.010750  28407.540217   
2009-10-27  37530.928377  30036.518070  24894.218373  27826.995523   
2009-10-28  34951.463648  29120.829939  23232.722800  26503.381895   
2009-10-29  37595.405862  30664.709376  24732.294859  27912.153131   
2009-10-30  36383.053943  29791.626280  23584.737252  27122.629395   
2009-11-03  37505.130312  30345.292056  24823.822389  27517.383467   
2009-11-04  38627.171326  30114.374794  25626.396111  27633.499682   
2009-11-05  38627.17

In [37]:
inicio_2010 = '2010-01-01'
fim_2010 = '2010-12-31'

carteira_geral10 = carteira.loc[inicio_2010:fim_2010]

carteira_normalizada_2010 = pd.DataFrame()

for coluna in carteira_geral1.columns:
    carteira_normalizada_2010[coluna] = (carteira_geral10[coluna] / carteira_geral10[coluna].iloc[0])* valores_2009[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2010['saldo'] = carteira_normalizada_2010.sum(axis=1)
print(len(carteira_normalizada_2010.columns))
valores_ult_2010 = (carteira_normalizada_2010['saldo'][-1] * 0.6) / 5
valores_ult_2010_selic = carteira_normalizada_2010['saldo'][-1] * 0.4
valores_2010 = {
    'BBAS3.SA': valores_ult_2010,
    'BBDC3.SA': valores_ult_2010,
    'ITUB4.SA': valores_ult_2010,
    'PETR4.SA': valores_ult_2010,
    'VALE3.SA': valores_ult_2010,
    'SELIC': valores_ult_2010_selic
}
print(carteira_normalizada_2010)
print(valores_ult_2010)

7
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2010-01-04  23589.697864  23589.697864  23589.697864  23589.697864   
2010-01-05  23352.999867  23264.795050  23742.557758  23387.426361   
2010-01-06  23384.561482  23127.750386  23536.754791  23703.468328   
2010-01-07  23392.458946  23074.473216  23295.699329  23482.240007   
2010-01-08  23526.584045  23104.916675  22942.903750  23355.827441   
...                  ...           ...           ...           ...   
2010-12-23  27693.091459  24311.285072  23311.584247  16760.971094   
2010-12-27  27523.571321  23884.775370  22908.509719  16956.238787   
2010-12-28  27470.037531  23790.006099  22896.675333  17353.299947   
2010-12-29  27657.399913  24074.338461  23406.427922  17555.077584   
2010-12-30  28032.105851  24358.684213  23584.232085  17763.373607   

                VALE3.SA         SELIC          saldo  
Date                           

In [38]:
inicio_2011 = '2011-01-01'
fim_2011 = '2011-12-31'

carteira_geral11 = carteira.loc[inicio_2011:fim_2011]
carteira_normalizada_2011 = pd.DataFrame()

for coluna in carteira_geral11.columns:
    carteira_normalizada_2011[coluna] = (carteira_geral11[coluna] / carteira_geral11[coluna].iloc[0])* valores_2010[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2011['saldo'] = carteira_normalizada_2011.sum(axis=1)
print(len(carteira_normalizada_2011.columns))
valores_ult_2011 = (carteira_normalizada_2011['saldo'][-1] * 0.6) / 5
valores_ult_2011_selic = carteira_normalizada_2011['saldo'][-1] * 0.4
valores_2011 = {
    'BBAS3.SA': valores_ult_2011,
    'BBDC3.SA': valores_ult_2011,
    'ITUB4.SA': valores_ult_2011,
    'PETR4.SA': valores_ult_2011,
    'VALE3.SA': valores_ult_2011,
    'SELIC': valores_ult_2011_selic
}
print(carteira_normalizada_2011)
print(valores_ult_2011)

7
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2011-01-03  25870.476096  25870.476096  25870.476096  25870.476096   
2011-01-04  25977.567003  25953.253106  26079.112549  25774.662843   
2011-01-05  26348.337674  26152.110749  26470.306506  26081.268359   
2011-01-06  26117.644229  25525.648332  25844.406854  25975.875334   
2011-01-07  25499.707598  25326.760268  25211.978970  25611.770993   
...                  ...           ...           ...           ...   
2011-12-23  22712.114723  26233.465006  23170.729612  22112.782227   
2011-12-26  22674.145359  26068.152013  23043.819822  22063.023957   
2011-12-27  22825.998912  26119.815687  23109.852034  22092.878608   
2011-12-28  22275.523533  25778.849009  22584.445889  21326.599382   
2011-12-29  22493.819128  26130.142338  22894.291772  21386.304024   

                VALE3.SA         SELIC          saldo  
Date                           

In [39]:
inicio_2012 = '2012-01-01'
fim_2012 = '2012-12-31'

carteira_geral12 = carteira.loc[inicio_2012:fim_2012]
carteira_normalizada_2012 = pd.DataFrame()

for coluna in carteira_geral12.columns:
    carteira_normalizada_2012[coluna] = (carteira_geral12[coluna] / carteira_geral12[coluna].iloc[0])* valores_2011[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2012['saldo'] = carteira_normalizada_2012.sum(axis=1)
print(len(carteira_normalizada_2012.columns))
valores_ult_2012 = (carteira_normalizada_2012['saldo'][-1] * 0.6) / 5
valores_ult_2012_selic = carteira_normalizada_2012['saldo'][-1] * 0.4
valores_2012 = {
    'BBAS3.SA': valores_ult_2012,
    'BBDC3.SA': valores_ult_2012,
    'ITUB4.SA': valores_ult_2012,
    'PETR4.SA': valores_ult_2012,
    'VALE3.SA': valores_ult_2012,
    'SELIC': valores_ult_2012_selic
}
print(carteira_normalizada_2012)
print(valores_ult_2012)

7
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2012-01-02  23976.296464  23976.296464  23976.296464  23976.296464   
2012-01-03  23867.711551  24075.400058  24575.184924  24929.996366   
2012-01-04  23749.277062  24075.400058  24603.347908  25107.995937   
2012-01-05  23502.488443  23895.298500  24420.166707  24818.762781   
2012-01-06  22910.236891  24255.475871  24603.347908  24841.002608   
...                  ...           ...           ...           ...   
2012-12-20  28331.965408  33795.509809  24471.465563  23780.850176   
2012-12-21  28558.620933  34017.915764  24608.631322  23374.141884   
2012-12-26  28331.965408  33480.745622  24580.932731  22662.412275   
2012-12-27  28399.960935  33313.898783  24172.937430  21928.086326   
2012-12-28  29011.936502  33304.108999  24311.364407  22052.359309   

                VALE3.SA         SELIC          saldo  
Date                           

In [40]:
#ENTRADA 6 etapa 1
inicio_2013_1 = '2013-01-01'
fim_2013_1 = '2013-02-18'

carteira_entrada_2013 = carteira.loc[inicio_2013_1:fim_2013_1]
carteira_normalizada_20131 = pd.DataFrame()

for coluna in carteira_entrada_2013.columns:
    carteira_normalizada_20131[coluna] = (carteira_entrada_2013[coluna] / carteira_entrada_2013[coluna].iloc[0])*valores_2012[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20131['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[5]
carteira_normalizada_20131['saldo'] = carteira_normalizada_20131.sum(axis=1)
print(carteira_normalizada_20131)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-01-02  22693.882443  22693.882443  22693.882443  22693.882443   
2013-01-03  23142.477169  23940.056145  23371.813206  23512.197762   
2013-01-04  22861.004696  23881.820253  23233.583077  23546.773624   
2013-01-07  23001.747072  23318.917136  22838.675535  23143.377437   
2013-01-08  23265.631792  23241.287663  22930.825135  22474.893782   
2013-01-09  23485.529586  23758.897612  23299.396036  22682.355984   
2013-01-10  23133.681538  23610.083868  23003.214806  22855.240566   
2013-01-11  22852.217836  23357.729751  23062.457927  22912.864075   
2013-01-14  23661.445716  23713.597373  23365.236493  22728.453034   
2013-01-15  23485.529586  23435.373365  23055.876631  22843.712350   
2013-01-16  23626.263192  23616.538968  23259.917427  22866.767025   
2013-01-17  23582.276265  23784.788714  23654.815803  22716.933603   
2013-01-18  23045.73

In [41]:
#ENTRADA 6 etapa 2
inicio_2013_2 = '2013-02-18'
fim_2013 = '2013-12-16'

carteira_geral13 = carteira.loc[inicio_2013_2:fim_2013]
carteira_normalizada_2013 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2013.columns:
    if coluna != 'saldo':
        carteira_normalizada_2013[coluna] *= carteira_normalizada_20131[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2013['saldo'] = carteira_normalizada_2013.sum(axis=1)

valores_ult_2013 = (carteira_normalizada_2013['saldo'][-1] * 0.6) / 5
valores_ult_2013_selic = carteira_normalizada_2013['saldo'][-1] * 0.4
valores_2013 = {
    'BBAS3.SA': valores_ult_2013,
    'BBDC3.SA': valores_ult_2013,
    'ITUB4.SA': valores_ult_2013,
    'PETR4.SA': valores_ult_2013,
    'VALE3.SA': valores_ult_2013,
    'SELIC': valores_ult_2013_selic
}
print(carteira_normalizada_2013)
print(valores_ult_2013)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-02-18  23333.660221  23661.907080  23408.342606  20630.799186   
2013-02-19  23769.704123  23247.014532  22954.015495  20861.310791   
2013-02-20  23546.836424  22786.759527  22552.354791  20308.086453   
2013-02-21  24506.138032  22741.370204  22387.742191  19962.313774   
2013-02-22  24922.825996  22987.708987  23112.044508  19766.379788   
...                  ...           ...           ...           ...   
2013-12-10  28455.102974  22695.108417  23423.945685  20323.608516   
2013-12-11  27680.115655  22319.987204  22799.911885  19712.729043   
2013-12-12  28272.122112  22291.118129  22993.063295  19806.710095   
2013-12-13  28497.722368  22377.690002  23082.210980  20229.632736   
2013-12-16  28545.232924  22435.388557  23156.498045  20370.605192   

                VALE3.SA          SELIC          saldo  
Date                            

In [43]:
#Entrada 7 - Dupla entrada no mesmo ano
inicio_2013_22 = '2013-12-16'
fim_2013_22 = '2013-12-31'
carteira_geral13_2 = carteira.loc[inicio_2013_22:fim_2013_22]
carteira_normalizada_2013_22 = carteira_geral13_2 / carteira_geral13_2.iloc[0]
for coluna in carteira_normalizada_2013.columns:
    if coluna == 'BBAS3.SA':
        carteira_normalizada_2013_22[coluna] *= carteira_normalizada_2013[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[6]
    elif coluna != 'saldo':
        carteira_normalizada_2013_22[coluna] *= carteira_normalizada_2013[coluna].iloc[-1]
carteira_normalizada_2013_22['saldo'] = carteira_normalizada_2013_22.sum(axis=1)

valores_ult_2013_22 = (carteira_normalizada_2013_22['saldo'][-1] * 0.6) / 5
valores_ult_2013_22_selic = carteira_normalizada_2013_22['saldo'][-1] * 0.4
valores_2013_22 = {
    'BBAS3.SA': valores_ult_2013_22,
    'BBDC3.SA': valores_ult_2013_22,
    'ITUB4.SA': valores_ult_2013_22,
    'PETR4.SA': valores_ult_2013_22,
    'VALE3.SA': valores_ult_2013_22,
    'SELIC': valores_ult_2013_22_selic
}
print(carteira_normalizada_2013_22)
print(valores_ult_2013_22)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2013-12-16  29111.732924  22435.388557  23156.498045  20370.605192   
2013-12-17  29135.938948  22579.685852  23186.216996  19736.224745   
2013-12-18  29305.477689  22514.754474  23327.364251  19783.212635   
2013-12-19  29547.675889  22846.593285  23847.392036  20264.872775   
2013-12-20  29668.761193  22695.108417  23282.793846  19865.449350   
2013-12-23  29692.986926  22622.978152  23532.234584  20041.663603   
2013-12-26  29608.208687  22617.549646  23696.746357  19877.198959   
2013-12-27  29402.360910  22814.343237  23629.469106  19888.945053   
2013-12-30  29547.675889  23288.117895  23980.903889  20065.161063   

                VALE3.SA          SELIC          saldo  
Date                                                    
2013-12-16  18230.683865  103973.390482  217278.299065  
2013-12-17  18167.070081  103973.390482  216778.527104  
2

In [44]:
#ENTRADA 8 etapa 1
inicio_2014_1 = '2014-01-01'
fim_2014_1 = '2014-09-15'

carteira_entrada_2014 = carteira.loc[inicio_2014_1:fim_2014_1]
carteira_normalizada_20141 = pd.DataFrame()

for coluna in carteira_entrada_2014.columns:
    carteira_normalizada_20141[coluna] = (carteira_entrada_2014[coluna] / carteira_entrada_2014[coluna].iloc[0])*valores_2013_22[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20141['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[7]
carteira_normalizada_20141['saldo'] = carteira_normalizada_20141.sum(axis=1)
print(carteira_normalizada_20141)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2014-01-02  26374.282621  26374.282621  26374.282621  26374.282621   
2014-01-03  26649.015030  25782.617968  26324.085915  25980.644978   
2014-01-06  26253.399288  25957.328411  26365.924620  26169.599054   
2014-01-07  25802.837065  25466.479027  26165.094222  25445.284123   
2014-01-08  25286.339779  25391.585380  26575.107893  25492.522053   
...                  ...           ...           ...           ...   
2014-09-09  38573.900337  32626.050397  36891.854912  36063.767007   
2014-09-10  37965.783253  32236.842162  36493.531165  35173.933270   
2014-09-11  38240.032765  32245.310644  36436.650108  35610.459035   
2014-09-12  37707.841339  31238.432255  35127.875745  33813.985805   
2014-09-15  40713.681165  32312.974627  36019.336961  34552.725520   

                VALE3.SA         SELIC          saldo  
Date                             

In [45]:
#ENTRADA 8 etapa 2
inicio_2014_2 = '2014-09-15'
fim_2014 = '2014-10-20'

carteira_geral13 = carteira.loc[inicio_2014_2:fim_2014]
carteira_normalizada_2014 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2014.columns:
    if coluna != 'saldo':
        carteira_normalizada_2014[coluna] *= carteira_normalizada_20141[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2014['saldo'] = carteira_normalizada_2014.sum(axis=1)

print(carteira_normalizada_2014)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2014-09-15  40713.681165  32312.974627  36019.336961  34552.725520   
2014-09-16  42506.089903  32871.404115  36948.735969  36181.308392   
2014-09-17  42506.089903  32803.727228  37110.008748  37138.296859   
2014-09-18  41840.347647  32126.832540  36673.713891  35744.773206   
2014-09-19  41059.349894  31856.083052  35801.244637  35106.773829   
2014-09-22  39113.279693  31729.171952  35317.562141  34569.512436   
2014-09-23  38588.369337  31458.419238  35146.839515  33797.196533   
2014-09-24  38409.123951  31788.409391  35421.882098  33965.089247   
2014-09-25  37410.486127  31001.521212  34312.254230  33310.305309   
2014-09-26  38191.453799  32245.310644  35753.795486  35157.148709   
2014-09-29  34926.691048  29994.639597  33250.078077  31228.409755   
2014-09-30  32391.686761  29613.890166  32102.555988  30372.147496   
2014-10-01  32903.81

In [46]:
#Entrada 9 - Dupla entrada no mesmo ano
inicio_2014_22 = '2014-10-20'
fim_2014_22 = '2014-12-31'
carteira_geral13 = carteira.loc[inicio_2014_22:fim_2014_22]
carteira_normalizada_2014_22 = carteira_geral13 / carteira_geral13.iloc[0]
for coluna in carteira_normalizada_2014.columns:
    if coluna == 'BBAS3.SA':
        carteira_normalizada_2014_22[coluna] *= carteira_normalizada_2014[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[8]
    elif coluna != 'saldo':
        carteira_normalizada_2014_22[coluna] *= carteira_normalizada_2014[coluna].iloc[-1]
carteira_normalizada_2014_22['saldo'] = carteira_normalizada_2014_22.sum(axis=1)
print(carteira_normalizada_2014_22)
valores_ult_2014_22 = (carteira_normalizada_2014_22['saldo'][-1] * 0.6) / 5
valores_ult_2014_22_selic = carteira_normalizada_2014_22['saldo'][-1] * 0.4
valores_2014_22 = {
    'BBAS3.SA': valores_ult_2014_22,
    'BBDC3.SA': valores_ult_2014_22,
    'ITUB4.SA': valores_ult_2014_22,
    'PETR4.SA': valores_ult_2014_22,
    'VALE3.SA': valores_ult_2014_22,
    'SELIC': valores_ult_2014_22_selic
}
print(carteira_normalizada_2014_22)
print(valores_ult_2014_22)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2014-10-20  41139.198016  30417.692728  33966.931132  30086.727528   
2014-10-21  38636.747880  28724.512377  32259.094873  28374.205364   
2014-10-22  37743.005304  28724.512377  32050.347312  27887.307075   
2014-10-23  34305.586978  27522.371748  30598.710329  26040.461320   
2014-10-24  35405.558581  27844.070853  31452.633584  27904.098701   
2014-10-27  33576.841758  26963.626103  30759.995923  24378.306970   
2014-10-28  35955.548421  28157.304702  32164.217074  25234.559809   
2014-10-29  35006.818168  27336.126426  31234.397731  23538.833982   
2014-10-30  37468.014423  29452.582509  33758.196386  24042.514478   
2014-10-31  38128.007884  30883.297968  34849.332080  25654.308079   
2014-11-03  37633.013797  30104.436297  34740.178271  24932.357635   
2014-11-04  39530.470264  30730.220063  35404.602231  24881.985111   
2014-11-05  36395.55

In [49]:
inicio_2015 = '2015-01-01'
fim_2015 = '2015-12-31'

carteira_geral15 = carteira.loc[inicio_2015:fim_2015]
carteira_normalizada_2015 = pd.DataFrame()

for coluna in carteira_geral15.columns:
    carteira_normalizada_2015[coluna] = (carteira_geral15[coluna] / carteira_geral15[coluna].iloc[0])* valores_2014_22[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2015['saldo'] = carteira_normalizada_2015.sum(axis=1)
print(len(carteira_normalizada_2015.columns))
valores_ult_2015 = (carteira_normalizada_2015['saldo'][-1] * 0.6) / 5
valores_ult_2015_selic = carteira_normalizada_2015['saldo'][-1] * 0.4
valores_2015 = {
    'BBAS3.SA': valores_ult_2015,
    'BBDC3.SA': valores_ult_2015,
    'ITUB4.SA': valores_ult_2015,
    'PETR4.SA': valores_ult_2015,
    'VALE3.SA': valores_ult_2015,
    'SELIC': valores_ult_2015_selic
}
print(carteira_normalizada_2015)
print(valores_ult_2015)

7
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2015-01-02  28003.871446  28003.871446  28003.871446  28003.871446   
2015-01-05  27422.777084  27658.698625  28144.593778  25610.374842   
2015-01-06  27806.054819  28396.244138  28599.883867  24772.649457   
2015-01-07  29030.073875  29650.112590  29634.619249  25939.478395   
2015-01-08  29128.979551  29715.663879  30098.157169  27614.927067   
...                  ...           ...           ...           ...   
2015-12-22  22668.279486  21867.179955  25451.486055  20314.772393   
2015-12-23  22441.602844  22127.251370  25355.792904  20733.632987   
2015-12-28  22638.060504  22293.703269  25260.106453  20045.505467   
2015-12-29  22366.035173  22262.501873  25279.250890  20015.585055   
2015-12-30  22275.365923  21326.220865  25193.138890  20045.505467   

                VALE3.SA          SELIC          saldo  
Date                          

In [50]:
#ENTRADA 10 etapa 1
inicio_2016_1 = '2016-01-01'
fim_2016_1 = '2016-12-19'

carteira_entrada_2016 = carteira.loc[inicio_2016_1:fim_2016_1]
carteira_normalizada_20161 = pd.DataFrame()

for coluna in carteira_entrada_2016.columns:
    carteira_normalizada_20161[coluna] = (carteira_entrada_2016[coluna] / carteira_entrada_2016[coluna].iloc[0])*valores_2015[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20161['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[9]
carteira_normalizada_20161['saldo'] = carteira_normalizada_20161.sum(axis=1)
print(carteira_normalizada_20161)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2016-01-04  26282.596486  26282.596486  26282.596486  26282.596486   
2016-01-05  26374.891258  26487.378595  26502.226492  25555.712785   
2016-01-06  26374.891258  26018.572024  26397.641240  24484.514199   
2016-01-07  25747.353372  25185.128108  25895.638860  23948.921614   
2016-01-08  25747.353372  25133.048288  25906.093481  23987.175544   
...                  ...           ...           ...           ...   
2016-12-13  52376.802993  41259.801852  39225.291078  59107.143597   
2016-12-14  51103.257532  40726.265998  39141.173461  56390.907959   
2016-12-15  53064.080728  40755.913057  38985.000690  56811.730704   
2016-12-16  53407.747509  41289.456579  38288.208094  56467.415819   
2016-12-19  52215.076187  40444.691797  37254.998708  56501.129824   

                VALE3.SA         SELIC          saldo  
Date                             

In [51]:
#ENTRADA 10 etapa 2
inicio_2016_2 = '2016-12-19'
fim_2016 = '2016-12-30'

carteira_geral16 = carteira.loc[inicio_2016_2:fim_2016]
carteira_normalizada_2016 = carteira_geral16 / carteira_geral16.iloc[0]
for coluna in carteira_normalizada_2016.columns:
    if coluna != 'saldo':
        carteira_normalizada_2016[coluna] *= carteira_normalizada_20161[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação


carteira_normalizada_2016['saldo'] = carteira_normalizada_2016.sum(axis=1)


valores_ult_2016 = (carteira_normalizada_2016['saldo'][-1] * 0.6) / 5
valores_ult_2016_selic = carteira_normalizada_2016['saldo'][-1] * 0.4
valores_2016 = {
    'BBAS3.SA': valores_ult_2016,
    'BBDC3.SA': valores_ult_2016,
    'ITUB4.SA': valores_ult_2016,
    'PETR4.SA': valores_ult_2016,
    'VALE3.SA': valores_ult_2016,
    'SELIC': valores_ult_2016_selic
}
print(carteira_normalizada_2016)
print(valores_ult_2016)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2016-12-19  52215.076187  40444.691797  37254.998708  56501.129824   
2016-12-20  53064.080728  40755.913057  38204.102677  56461.784087   
2016-12-21  53064.080728  40014.878469  38192.089011  56422.438350   
2016-12-22  52558.717201  39950.139600  38504.461392  55124.017985   
2016-12-23  54054.620381  40907.752621  39591.363859  56028.980978   
2016-12-26  54903.650689  41745.657782  39908.374334  56737.220805   
2016-12-27  55287.718037  41850.396407  39884.000549  56697.875068   
2016-12-28  55975.043009  42822.970992  40822.876800  58153.672862   
2016-12-29  56783.629805  43601.026059  41274.021167  58507.790015   

                VALE3.SA        SELIC          saldo  
Date                                                  
2016-12-19  54269.843632  84701.76406  325387.504207  
2016-12-20  55373.991377  84701.76406  328561.635986  
2016-12-2

In [55]:
#ENTRADA 11 etapa 1
inicio_2017_1 = '2017-01-01'
fim_2017_1 = '2017-11-21'

carteira_entrada_2017 = carteira.loc[inicio_2017_1:fim_2017_1]
carteira_normalizada_20171 = pd.DataFrame()

for coluna in carteira_entrada_2017.columns:
    carteira_normalizada_20171[coluna] = (carteira_entrada_2017[coluna] / carteira_entrada_2017[coluna].iloc[0])*valores_2016[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20171['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[10]
carteira_normalizada_20171['saldo'] = carteira_normalizada_20171.sum(axis=1)
print(carteira_normalizada_20171)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2017-01-02  40604.070906  40604.070906  40604.070906  40604.070906   
2017-01-03  42461.771469  42249.561383  42323.659933  42930.634573   
2017-01-04  42240.613359  42235.480729  42505.318413  42930.634573   
2017-01-05  42137.432133  42475.129847  43086.571297  43623.063913   
2017-01-06  42594.475105  42799.378421  43231.894206  43373.792148   
...                  ...           ...           ...           ...   
2017-11-15  47110.279293  47091.526356  51214.316896  42515.181632   
2017-11-16  49342.631943  48127.201769  52503.427981  43789.248974   
2017-11-17  50173.969952  49319.778991  53229.330443  44370.898633   
2017-11-20  50173.969952  49319.778991  53229.330443  44370.898633   
2017-11-21  54151.895818  50167.156348  54293.171642  44038.520739   

                VALE3.SA          SELIC          saldo  
Date                            

In [56]:
#ENTRADA 11 etapa 2
inicio_2017_2 = '2017-11-21'
fim_2017 = '2017-12-30'

carteira_geral17 = carteira.loc[inicio_2017_2:fim_2017]
carteira_normalizada_2017 = carteira_geral17 / carteira_geral17.iloc[0]
for coluna in carteira_normalizada_2017.columns:
    if coluna != 'saldo':
        carteira_normalizada_2017[coluna] *= carteira_normalizada_20171[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação


carteira_normalizada_2017['saldo'] = carteira_normalizada_2017.sum(axis=1)


valores_ult_2017 = (carteira_normalizada_2017['saldo'][-1] * 0.6) / 5
valores_ult_2017_selic = carteira_normalizada_2017['saldo'][-1] * 0.4
valores_2017 = {
    'BBAS3.SA': valores_ult_2017,
    'BBDC3.SA': valores_ult_2017,
    'ITUB4.SA': valores_ult_2017,
    'PETR4.SA': valores_ult_2017,
    'VALE3.SA': valores_ult_2017,
    'SELIC': valores_ult_2017_selic
}
print(carteira_normalizada_2017)
print(valores_ult_2017)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2017-11-21  54151.895818  50167.156348  54293.171642  44038.520739   
2017-11-22  54212.255611  49586.553224  53767.514460  44620.158742   
2017-11-23  54278.497517  49241.328086  53604.810931  44841.745300   
2017-11-24  53980.453803  48645.034060  53104.163822  44592.473535   
2017-11-27  53251.875917  48958.866561  53429.590256  43955.434036   
2017-11-28  54046.669123  48723.503016  53517.205438  43872.335677   
2017-11-29  51596.030983  48174.256426  52415.817643  42459.784021   
2017-11-30  49559.354076  47672.133090  51652.339520  42598.268335   
2017-12-01  50966.814295  47954.601837  51621.041810  43235.303949   
2017-12-04  52208.704642  48264.307623  52384.776669  42875.244733   
2017-12-05  51447.020757  47651.968872  52071.755968  42404.390296   
2017-12-06  53003.471262  48311.391163  52622.655739  42986.028299   
2017-12-07  50950.25

In [57]:
#ENTRADA 12, 14 E 15 etapa 1
inicio_2018_1 = '2018-01-01'
fim_2018_1 = '2018-03-19'

carteira_entrada_2018 = carteira.loc[inicio_2018_1:fim_2018_1]
carteira_normalizada_20181 = pd.DataFrame()

for coluna in carteira_entrada_2018.columns:
    carteira_normalizada_20181[coluna] = (carteira_entrada_2018[coluna] / carteira_entrada_2018[coluna].iloc[0])*valores_2017[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20181['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[11]
carteira_normalizada_20181['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[13]
carteira_normalizada_20181['BBDC3.SA'].iloc[-1] += acoes_num['BBDC3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[14]
carteira_normalizada_20181['saldo'] = carteira_normalizada_20181.sum(axis=1)
print(carteira_normalizada_20181)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2018-01-02  40959.913081  40959.913081  40959.913081  40959.913081   
2018-01-03  41482.327438  41134.161716  41239.947679  41331.164042   
2018-01-04  41880.344301  41883.212391  42098.733637  41405.400348   
2018-01-05  41880.344301  41895.909565  42220.083162  41652.889417   
2018-01-08  41979.855345  41895.909565  42080.062266  42147.881442   
2018-01-09  41581.821102  41578.525783  41622.671223  42147.881442   
2018-01-10  41407.678016  41311.893671  41482.671912  41578.649640   
2018-01-11  42353.020517  41895.909565  42350.764775  42692.360866   
2018-01-12  42452.531561  41895.909565  42313.440020  42816.105400   
2018-01-15  42663.988184  41997.444235  42574.788856  42939.849935   
2018-01-16  43037.138459  42695.737544  42966.848086  43682.334500   
2018-01-17  44057.073274  44003.392676  43676.263072  45439.527721   
2018-01-18  43758.56

In [58]:
#ENTRADA 12, 14 E 15 etapa 2
inicio_2018_2 = '2018-03-19'
fim_2018 = '2018-06-18'

carteira_geral18 = carteira.loc[inicio_2018_2:fim_2018]
carteira_normalizada_2018 = carteira_geral18 / carteira_geral18.iloc[0]
for coluna in carteira_normalizada_2018.columns:
    if coluna != 'saldo':
        carteira_normalizada_2018[coluna] *= carteira_normalizada_20181[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2018['saldo'] = carteira_normalizada_2018.sum(axis=1)
valores_ult_2018 = (carteira_normalizada_2018['saldo'][-1] * 0.6) / 5
valores_ult_2018_selic = carteira_normalizada_2018['saldo'][-1] * 0.4
valores_2018 = {
    'BBAS3.SA': valores_ult_2018,
    'BBDC3.SA': valores_ult_2018,
    'ITUB4.SA': valores_ult_2018,
    'PETR4.SA': valores_ult_2018,
    'VALE3.SA': valores_ult_2018,
    'SELIC': valores_ult_2018_selic
}
print(carteira_normalizada_2018)
print(valores_ult_2018)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2018-03-19  57058.563494  48297.879786  48768.143968  51878.273173   
2018-03-20  57279.074583  48218.121895  48923.825073  52448.346932   
2018-03-21  58230.037458  48656.830520  48729.211100  54654.352698   
2018-03-22  57182.612494  49388.018513  48573.529996  53836.405817   
2018-03-23  56782.921924  49986.222061  48807.048055  53910.768101   
...                  ...           ...           ...           ...   
2018-06-12  37928.641410  35756.761222  38840.665638  38396.649730   
2018-06-13  37240.052166  35126.868461  38460.827342  37676.405045   
2018-06-14  35553.712818  33325.124392  37097.309960  37502.556956   
2018-06-15  35132.118500  34174.723071  37457.674627  37179.678338   
2018-06-18  34457.593597  33251.871208  36697.998035  36012.383788   

                VALE3.SA          SELIC          saldo  
Date                            

In [59]:
#ENTRADA 13 - Dupla Entrada mesmo ano
inicio_2018_22 = '2018-06-18'
fim_2018_22 = '2018-12-31'
carteira_geral182 = carteira.loc[inicio_2018_22:fim_2018_22]
carteira_normalizada_2018_22 = carteira_geral182 / carteira_geral182.iloc[0]
for coluna in carteira_normalizada_2018.columns:
    if coluna == 'VALE3.SA':
        carteira_normalizada_2018_22[coluna] *= carteira_normalizada_2018[coluna].iloc[-1] + acoes_num['VALE3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[12]
    elif coluna != 'saldo':
        carteira_normalizada_2018_22[coluna] *= carteira_normalizada_2018[coluna].iloc[-1]
carteira_normalizada_2018_22['saldo'] = carteira_normalizada_2018_22.sum(axis=1)
valores_ult_2018_22 = (carteira_normalizada_2018_22['saldo'][-1] * 0.6) / 5
valores_ult_2018_22_selic = carteira_normalizada_2018_22['saldo'][-1] * 0.4
valores_2018_22 = {
    'BBAS3.SA': valores_ult_2018_22,
    'BBDC3.SA': valores_ult_2018_22,
    'ITUB4.SA': valores_ult_2018_22,
    'PETR4.SA': valores_ult_2018_22,
    'VALE3.SA': valores_ult_2018_22,
    'SELIC': valores_ult_2018_22_selic
}
print(valores_ult_2018_22)
print(carteira_normalizada_2018_22)

49298.214807956756
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2018-06-18  34457.593597  33251.871208  36697.998035  36012.383788   
2018-06-19  36874.685412  34716.714395  38353.695401  38297.309952   
2018-06-20  37942.697624  34936.450110  39064.668133  40259.367929   
2018-06-21  36551.465630  33852.464483  38022.553770  37502.556956   
2018-06-22  36677.936341  34570.234844  38314.740948  37527.388423   
...                  ...           ...           ...           ...   
2018-12-20  65110.187141  49607.718472  51476.212361  53719.765369   
2018-12-21  65124.961200  49252.295714  51238.441977  53869.765931   
2018-12-26  65553.322221  49076.731216  50123.915789  56058.738742   
2018-12-27  66646.380443  50315.783918  50970.970082  56032.885088   
2018-12-28  68670.009331  51146.795167  52754.186800  58644.493515   

                VALE3.SA          SELIC          saldo  
Date         

In [60]:
#ENTRADA 16 etapa 1
inicio_2019_1 = '2019-01-01'
fim_2019_1 = '2019-03-18'

carteira_entrada_2019 = carteira.loc[inicio_2019_1:fim_2019_1]
carteira_normalizada_20191 = pd.DataFrame()

for coluna in carteira_entrada_2019.columns:
    carteira_normalizada_20191[coluna] = (carteira_entrada_2019[coluna] / carteira_entrada_2019[coluna].iloc[0])*valores_2018_22[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20191['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[15]
carteira_normalizada_20191['saldo'] = carteira_normalizada_20191.sum(axis=1)
print(carteira_normalizada_20191)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2019-01-02  49298.214808  49298.214808  49298.214808  49298.214808   
2019-01-03  49501.086635  50328.586586  50110.972299  50507.116041   
2019-01-04  49501.086635  49225.474113  49271.547200  50650.539958   
2019-01-07  49176.502129  49639.120907  49364.827403  51449.634170   
2019-01-08  48618.586003  50190.720629  49991.037385  51142.288547   
2019-01-09  48486.708898  51238.622577  50750.524146  52207.765863   
2019-01-10  49196.771454  51390.306114  50404.099960  51756.985303   
2019-01-11  49399.665603  51183.477282  50097.640107  51203.759875   
2019-01-14  50545.864637  51238.622577  50470.683537  50916.906532   
2019-01-15  49957.555686  51431.655573  49684.587205  50875.932993   
2019-01-16  49480.813590  51707.441845  49538.020151  50855.446224   
2019-01-17  49663.378888  51914.259806  49538.020151  51552.073527   
2019-01-18  49247.50

In [61]:
#ENTRADA 16 etapa 2
inicio_2019_2 = '2019-03-18'
fim_2019 = '2019-12-30'

carteira_geral19 = carteira.loc[inicio_2019_2:fim_2019]
carteira_normalizada_2019 = carteira_geral19 / carteira_geral19.iloc[0]
for coluna in carteira_normalizada_2019.columns:
    if coluna != 'saldo':
        carteira_normalizada_2019[coluna] *= carteira_normalizada_20191[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2019['saldo'] = carteira_normalizada_2019.sum(axis=1)

valores_ult_2019 = (carteira_normalizada_2019['saldo'][-1] * 0.6) / 5
valores_ult_2019_selic = carteira_normalizada_2019['saldo'][-1] * 0.4
valores_2019 = {
    'BBAS3.SA': valores_ult_2019,
    'BBDC3.SA': valores_ult_2019,
    'ITUB4.SA': valores_ult_2019,
    'PETR4.SA': valores_ult_2019,
    'VALE3.SA': valores_ult_2019,
    'SELIC': valores_ult_2019_selic
}
print(carteira_normalizada_2019)
print(valores_ult_2019)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2019-03-18  58010.161560  55328.196627  50551.173219  58887.405964   
2019-03-19  56770.181766  54237.939612  49371.714359  59829.924093   
2019-03-20  55497.847520  53216.645472  48260.889686  59358.656763   
2019-03-21  54333.306347  51546.740463  47410.582810  58518.589012   
2019-03-22  51378.886547  50180.441698  46011.711863  55322.179103   
...                  ...           ...           ...           ...   
2019-12-20  59374.757511  58262.497113  51104.135140  63186.822879   
2019-12-23  61165.535727  59434.056068  51580.154033  63687.977309   
2019-12-26  61863.236284  60063.533233  52364.190082  64544.118192   
2019-12-27  61595.758454  59678.858783  52224.200457  64611.733348   
2019-12-30  61421.338121  59573.927803  51944.179291  63891.941337   

                VALE3.SA          SELIC          saldo  
Date                            

In [62]:
inicio_2020 = '2020-01-01'
fim_2020 = '2020-12-31'

carteira_geral20 = carteira.loc[inicio_2020:fim_2020]
carteira_normalizada_2020 = pd.DataFrame()
for coluna in carteira_geral20.columns:
    carteira_normalizada_2020[coluna] = (carteira_geral20[coluna] / carteira_geral20[coluna].iloc[0])* valores_2019[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2020['saldo'] = carteira_normalizada_2020.sum(axis=1)
print(len(carteira_normalizada_2020.columns))
valores_ult_2020 = (carteira_normalizada_2020['saldo'][-1] * 0.6) / 5
valores_ult_2020_selic = carteira_normalizada_2020['saldo'][-1] * 0.4
valores_2020 = {
    'BBAS3.SA': valores_ult_2020,
    'BBDC3.SA': valores_ult_2020,
    'ITUB4.SA': valores_ult_2020,
    'PETR4.SA': valores_ult_2020,
    'VALE3.SA': valores_ult_2020,
    'SELIC': valores_ult_2020_selic
}
print(carteira_normalizada_2020)
print(valores_ult_2020)


7
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2020-01-02  48442.353068  48442.353068  48442.353068  48442.353068   
2020-01-03  48361.306602  47686.438926  47932.807087  48047.884462   
2020-01-06  47722.011152  47412.941263  47219.473794  48615.926976   
2020-01-07  47361.853590  46606.083853  46124.031596  48426.582209   
2020-01-08  46929.642487  45894.938939  45372.492397  48126.771612   
...                  ...           ...           ...           ...   
2020-12-22  37324.856383  37013.843285  40815.298971  43046.711040   
2020-12-23  37938.965094  37804.936284  41716.509733  44103.945879   
2020-12-28  38358.125889  38124.409939  42016.902569  44466.872767   
2020-12-29  38133.933849  38120.572940  41925.466423  44608.888529   
2020-12-30  37822.000512  37379.182453  41311.597501  44719.342039   

                VALE3.SA          SELIC          saldo  
Date                          

In [63]:
#ENTRADA 17 etapa 1
inicio_2021_1 = '2021-01-01'
fim_2021_1 = '2021-02-22'

carteira_entrada_2021 = carteira.loc[inicio_2021_1:fim_2021_1]
carteira_normalizada_20211 = pd.DataFrame()

for coluna in carteira_entrada_2021.columns:
    carteira_normalizada_20211[coluna] = (carteira_entrada_2021[coluna] / carteira_entrada_2021[coluna].iloc[0])*valores_2020[coluna]
carteira_normalizada_20211['VALE3.SA'].iloc[-1] += acoes_num['VALE3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[16]
carteira_normalizada_20211['saldo'] = carteira_normalizada_20211.sum(axis=1)
print(carteira_normalizada_20211)

                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2021-01-04  37564.754279  37564.754279  37564.754279  37564.754279   
2021-01-05  37175.011995  37385.143462  37321.613383  39033.040739   
2021-01-06  38024.454376  38324.975958  38415.740597  39111.000338   
2021-01-07  39533.436208  39264.760288  39911.022601  40280.431522   
2021-01-08  39763.280153  38962.113127  39898.869785  40436.357484   
2021-01-11  39113.719851  38372.740228  38999.269473  40098.522410   
2021-01-12  39473.478908  38388.657272  39120.841285  39799.667135   
2021-01-13  37524.782746  37671.873579  38476.518319  37876.602820   
2021-01-14  37434.848322  38866.536421  39619.267711  38266.414343   
2021-01-15  36275.621982  37432.947140  38123.969339  36538.252408   
2021-01-18  35905.865465  37592.227054  37929.458804  36473.282693   
2021-01-19  35296.279748  36891.338510  37479.654556  37278.892271   
2021-01-20  34506.80

In [64]:
#ENTRADA 17 etapa 2
inicio_2021_2 = '2021-02-22'
fim_2021 = '2021-07-16'

carteira_geral21 = carteira.loc[inicio_2021_2:fim_2021]
carteira_normalizada_2021 = carteira_geral21 / carteira_geral21.iloc[0]
for coluna in carteira_normalizada_2021.columns:
    if coluna != 'saldo':
        carteira_normalizada_2021[coluna] *= carteira_normalizada_20211[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2021['saldo'] = carteira_normalizada_2021.sum(axis=1)

print(carteira_normalizada_2021)


                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2021-02-22  28810.640628  32756.598357  31203.724577  28157.321590   
2021-02-23  31357.722665  34256.413051  33201.931323  31262.813737   
2021-02-24  31234.063596  34033.042404  32714.580191  31704.598325   
2021-02-25  30358.144423  33155.489996  31691.089348  30132.362352   
2021-02-26  28905.161799  32852.328322  31307.537728  28897.964835   
...                  ...           ...           ...           ...   
2021-07-12  35129.179331  38208.223235  36682.304306  37244.166904   
2021-07-13  34978.326313  38334.386062  36904.702213  37472.489236   
2021-07-14  35215.384543  38821.000976  36954.104572  37163.576185   
2021-07-15  34676.601968  38640.781501  36509.325125  36371.145879   
2021-07-16  34256.333248  37973.942734  35916.292228  35833.910496   

                VALE3.SA          SELIC          saldo  
Date                            

In [65]:
#ENTRADA 18 e 19 - Dupla entrada mesmo ano
inicio_2021_22 = '2021-10-20'
fim_2021_22 = '2021-12-31'
carteira_geral21 = carteira.loc[inicio_2021_22:fim_2021_22]
carteira_normalizada_2021_22 = carteira_geral21 / carteira_geral21.iloc[0]
for coluna in carteira_normalizada_2021.columns:
    if coluna == 'ITUB4.SA':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1] + acoes_num['ITUB4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[17]
    elif coluna == 'BBAS3.SA':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1] + acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[18]
    elif coluna != 'saldo':
        carteira_normalizada_2021_22[coluna] *= carteira_normalizada_2021[coluna].iloc[-1]
carteira_normalizada_2021_22['saldo'] = carteira_normalizada_2021_22.sum(axis=1)
valores_ult_2021_22 = (carteira_normalizada_2021_22['saldo'][-1] * 0.6) / 5
valores_ult_2021_22_selic = carteira_normalizada_2021_22['saldo'][-1] * 0.4
valores_2021_22 = {
    'BBAS3.SA': valores_ult_2021_22,
    'BBDC3.SA': valores_ult_2021_22,
    'ITUB4.SA': valores_ult_2021_22,
    'PETR4.SA': valores_ult_2021_22,
    'VALE3.SA': valores_ult_2021_22,
    'SELIC': valores_ult_2021_22_selic
}
print(valores_ult_2021_22)
print(carteira_normalizada_2021_22)

70405.07674992217
                BBAS3.SA      BBDC3.SA      ITUB4.SA      PETR4.SA  \
Date                                                                 
2021-10-20  34426.283248  37973.942734  36259.392228  35833.910496   
2021-10-21  32903.580253  36887.791262  35519.409185  34513.705427   
2021-10-22  31899.481907  35678.703139  34155.508683  34174.224124   
2021-10-25  32561.529178  36252.504348  34822.945556  36512.864139   
2021-10-26  32219.472075  35412.295702  34445.680609  36160.812442   
2021-10-27  32186.367149  36006.598767  34706.864034  36072.798771   
2021-10-28  31899.481907  35658.206249  34634.333804  36412.277086   
2021-10-29  31447.087109  34838.482078  33821.772713  34136.509955   
2021-11-01  32164.306241  35965.600019  35168.795933  35205.238428   
2021-11-03  32373.946656  35877.914047  35502.761440  33759.311495   
2021-11-04  31612.596666  33929.135222  33629.596775  32690.574058   
2021-11-05  32318.780823  35303.534308  33223.040467  32501.982298   
20

In [66]:
#ENTRADA 20 E 21 - etapa 1
inicio_2022_1 = '2022-01-01'
fim_2022_1 = '2022-09-16'

carteira_entrada_2022 = carteira.loc[inicio_2022_1:fim_2022_1]
carteira_normalizada20221 = pd.DataFrame()
for coluna in carteira_entrada_2022.columns:
    carteira_normalizada20221[coluna] = (carteira_entrada_2022[coluna] / carteira_entrada_2022[coluna].iloc[0])*valores_2021_22[coluna]
carteira_normalizada20221['ITUB4.SA'].iloc[-1] += acoes_num['ITUB4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[19]
carteira_normalizada20221['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[20]
carteira_normalizada20221['saldo'] = carteira_normalizada20221.sum(axis=1)
print(carteira_normalizada20221)

                 BBAS3.SA      BBDC3.SA      ITUB4.SA       PETR4.SA  \
Date                                                                   
2022-01-03   70405.076750  70405.076750  70405.076750   70405.076750   
2022-01-04   70478.351747  70734.066790  72401.696633   70671.305436   
2022-01-05   69305.759135  70265.617057  71026.984509   67936.422384   
2022-01-06   69867.632583  71032.155474  72467.137847   67888.012979   
2022-01-07   69940.901158  71330.234004  74070.989927   68202.656191   
...                   ...           ...           ...            ...   
2022-09-12  110462.745142  77564.159938  90200.856196  113821.001527   
2022-09-13  109341.245946  76464.976320  89268.210781  110820.007547   
2022-09-14  109449.996060  76034.870261  88935.113512  112519.379895   
2022-09-15  108335.368402  75700.336557  89168.280182  112302.436130   
2022-09-16  107621.853474  74935.681565  88937.122565  111290.042132   

                VALE3.SA          SELIC          saldo  
Date  

In [67]:
#ENTRADA 20 E 21 - ETAPA 2
inicio_2022_2 = '2022-09-16'
fim_2022 = '2022-12-31'

carteira_geral22 = carteira.loc[inicio_2022_2:fim_2022]
carteira_normalizada_2022 = carteira_geral22 / carteira_geral22.iloc[0]
for coluna in carteira_normalizada_2022.columns:
    if coluna != 'saldo':
        carteira_normalizada_2022[coluna] *= carteira_normalizada20221[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2022['saldo'] = carteira_normalizada_2022.sum(axis=1)
valores_ult_2022 = (carteira_normalizada_2022['saldo'][-1] * 0.6) / 5
valores_ult_2022_selic = carteira_normalizada_2022['saldo'][-1] * 0.4
valores_2022 = {
    'BBAS3.SA': valores_ult_2022,
    'BBDC3.SA': valores_ult_2022,
    'ITUB4.SA': valores_ult_2022,
    'PETR4.SA': valores_ult_2022,
    'VALE3.SA': valores_ult_2022,
    'SELIC': valores_ult_2022_selic
}
print(carteira_normalizada_2022)
print(valores_ult_2022)

                 BBAS3.SA      BBDC3.SA      ITUB4.SA       PETR4.SA  \
Date                                                                   
2022-09-16  107621.853474  74935.681565  88937.122565  111290.042132   
2022-09-19  109752.452219  76751.718769  92270.575044  113061.718828   
2022-09-20  111418.659753  79571.372312  95337.392190  112410.908012   
2022-09-21  110626.542688  79045.683745  94337.335872  112700.156126   
2022-09-22  112647.860767  80622.769754  96237.440748  115484.196100   
...                   ...           ...           ...            ...   
2022-12-23   99340.700597  64060.028788  85794.157342   97170.552185   
2022-12-26   99199.918501  62573.611125  84091.194608   96474.255473   
2022-12-27   95905.463882  62957.205236  83614.369866   97131.865621   
2022-12-28   97623.066108  64875.170717  85317.311317   95932.705016   
2022-12-29   97792.033014  64587.484018  85147.044132   94772.230976   

                VALE3.SA          SELIC          saldo  
Date  

In [68]:
#ENTRADA 22 E 23 etapa 1
inicio_2023_1 = '2023-01-01'
fim_2023_1 = '2023-07-21'

carteira_entrada_2023 = carteira.loc[inicio_2023_1:fim_2023_1]
carteira_normalizada_20231 = pd.DataFrame()
for coluna in carteira_entrada_2023.columns:
    carteira_normalizada_20231[coluna] = (carteira_entrada_2023[coluna] / carteira_entrada_2023[coluna].iloc[0])*valores_2022[coluna]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_20231['PETR4.SA'].iloc[-1] += acoes_num['PETR4.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[21]
carteira_normalizada_20231['BBAS3.SA'].iloc[-1] += acoes_num['BBAS3.SA'] * resultados_vendacoberta['ResultadosTotais'].iloc[22]
carteira_normalizada_20231['saldo'] = carteira_normalizada_20231.sum(axis=1)
print(carteira_normalizada_20231)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2023-01-02   92787.705136   92787.705136   92787.705136   92787.705136   
2023-01-03   91058.044893   88401.128548   90855.396913   90439.665899   
2023-01-04   92229.772285   89175.920772   91120.631462   93313.989054   
2023-01-05   96442.311389   92063.939103   93962.253315   96674.087192   
2023-01-06   97781.400097   94529.313703   95591.399158   96107.337208   
...                   ...            ...            ...            ...   
2023-07-17  148654.758607  109967.086181  111959.580802  140860.320775   
2023-07-18  149149.139037  109517.957391  109784.870442  140082.881471   
2023-07-19  146553.609817  109218.510888  108619.841683  141394.802929   
2023-07-20  148160.352606  109892.241313  110639.244319  141540.570790   
2023-07-21  150318.150467  113036.306700  112619.788478  144244.250843   

                VALE3.SA          SEL

In [69]:
#ENTRADA 22 E 23 etapa 2
inicio_2023_2 = '2023-07-21'
fim_2023 = '2023-12-31'

carteira_geral23 = carteira.loc[inicio_2023_2:fim_2023]
carteira_normalizada_2023 = carteira_geral23 / carteira_geral23.iloc[0]
for coluna in carteira_normalizada_2023.columns:
    if coluna != 'saldo':
        carteira_normalizada_2023[coluna] *= carteira_normalizada_20231[coluna].iloc[-1]
#                                                      acessando primeira linha/primeira operação
carteira_normalizada_2023['saldo'] = carteira_normalizada_2023.sum(axis=1)
valores_ult_2023 = (carteira_normalizada_2023['saldo'][-1] * 0.6) / 5
valores_ult_2023_selic = carteira_normalizada_2023['saldo'][-1] * 0.4
valores_2023 = {
    'BBAS3.SA': valores_ult_2023,
    'BBDC3.SA': valores_ult_2023,
    'ITUB4.SA': valores_ult_2023,
    'PETR4.SA': valores_ult_2023,
    'VALE3.SA': valores_ult_2023,
    'SELIC': valores_ult_2023_selic
}
print(carteira_normalizada_2023)
print(valores_ult_2023)

                 BBAS3.SA       BBDC3.SA       ITUB4.SA       PETR4.SA  \
Date                                                                     
2023-07-21  150318.150467  113036.306700  112619.788478  144244.250843   
2023-07-24  147374.994419  111089.982636  110833.402634  147257.450762   
2023-07-25  146600.464151  110491.097079  111066.387880  150659.438568   
2023-07-26  147839.710016  111389.406795  111687.752851  150659.438568   
2023-07-27  144803.598030  108919.071833  109202.356062  142834.866613   
...                   ...            ...            ...            ...   
2023-12-21  178477.120932  116479.572807  130019.387863  190704.494509   
2023-12-22  178970.219434  116769.091137  132081.284813  192538.703815   
2023-12-26  180613.966574  117234.612956  133429.461854  195630.656033   
2023-12-27  180350.982416  118088.078315  134341.449124  195787.860498   
2023-12-28  182093.339000  118553.600135  134698.328795  195158.999761   

                VALE3.SA          SEL